In [ ]:
# classes with samples between 1.5k to 10k

# classes = ['Lysozyme-PF03245',
#  'Lysozyme-PF16754',
#  'Lysozyme-PF11860',
#  'Lysozyme-PF13702',
#  'Lysozyme-PF00959',
#  'Lysozyme-PF00182',
#  'Lysozyme-PF00704',
#  'Lysozyme-PF01374',
#  'Lysozyme-PF05838',
#  'Lysozyme-PF18013',
#  'Lysozyme-PF04965',
#  'Lysozyme-PF01183',
#  'Lysozyme-PF00722',
#  'peptidase-PF05193',
#  'peptidase-PF01551',
#  'peptidase-PF00675',
#  'peptidase-PF01435',
#  'peptidase-PF01433',
#  'peptidase-PF10502',
#  'peptidase-PF00246',
#  'peptidase-PF03572',
#  'peptidase-PF00814',
#  'peptidase-PF17900',
#  'Amidase_2-PF01510',
#  'Amidase_3-PF01520',
#  'CHAP-PF05257',
#  'SH3_4-PF06347',
#  'SH3_3-PF08239',
#  'SH3_5-PF08460',
#  'LysM-PF01476']

classes= ['CHAP-PF05257','SH3_4-PF06347',]

In [3]:
%load_ext autoreload
%autoreload 
import sys
from pathlib import Path
ProjectRoot = Path().cwd().parent.parent
sys.path.append(str(ProjectRoot))

In [3]:
# from src.data.modelData import ObjectDetection
# #! trash-put /home/satish27may/ProteinDomainDetection/data/PfamData/images_224/
# #! trash-put /home/satish27may/ProteinDomainDetection/data/PfamData/annotations_224/
# data_handler = ObjectDetection(class_names=classes, img_dim=224)
# #data_handler.create_coco_data(augment_data=False, num_augs=1000, img_h=224, img_w=1000)

In [4]:
import torch
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm import tqdm

/home/satish27may/anaconda3/envs/detectron2/lib/python3.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [5]:
from detectron2.structures import BoxMode

def create_train_valid_test_data(data_df):
    train_dfs,valid_dfs = [],[],
    for class_name in data_df['Class'].unique():
        class_df = data_df[data_df['Class']==class_name].sample(frac=1)
        num_samples = class_df.shape[0]
        num_train_samples = int(round(num_samples*0.7))
        train_dfs.append(class_df.iloc[:num_train_samples,:])
        valid_dfs.append(class_df.iloc[num_train_samples:,:])
    return pd.concat(train_dfs,axis='rows').sample(frac=1), pd.concat(valid_dfs,axis='rows').sample(frac=1)
    
def create_dataset(img_h, img_w, mode, classes):
    model_data = pd.read_csv(ProjectRoot/'data/PfamData/model_data.csv')
    model_data = model_data[model_data['Class'].isin([x.split('-')[-1] for x in classes])]
    print(f"Classes selected: {model_data['Class'].unique()}")
    model_data = model_data.reset_index(drop=True)
    model_data['dom_pos'] = model_data['dom_pos'].apply(lambda x: [int(y) for y in x.replace('[','').replace(']','').split(',')])
    C2I = {class_name:index for index, class_name in enumerate(model_data['Class'].unique())}
    train_dicts_list = []
    valid_dicts_list = []
    train, valid = create_train_valid_test_data(model_data)
    train = train.reset_index(drop=True)
    valid = valid.reset_index(drop=True)
    for index in tqdm(range(train.shape[0])):
        x1,x2 = train['dom_pos'][index]
        train_dicts_list.append({'file_name':train['img_pth'][index],
                           'height':img_h,
                           'width': img_w,
                           'image_id': index,
                           'annotations':[{'bbox':[x1, 0, x2, img_h],
                                           'bbox_mode':BoxMode.XYXY_ABS,
                                           'category_id':  C2I[train['Class'][index]],
                                          }]
                          })
    for index in tqdm(range(valid.shape[0])):
        x1,x2 = valid['dom_pos'][index]
        valid_dicts_list.append({'file_name':valid['img_pth'][index],
                           'height':img_h,
                           'width': img_w,
                           'image_id': index,
                           'annotations':[{'bbox':[x1, 0, x2, img_h],
                                           'bbox_mode':BoxMode.XYXY_ABS,
                                           'category_id':  C2I[valid['Class'][index]],
                                          }]
                          })
    if mode=='train':return train_dicts_list
    elif mode=='valid': return valid_dicts_list

train_list = create_dataset(img_h=224, img_w=1000, mode='train', classes=classes)
valid_list = create_dataset(img_h=224, img_w=1000, mode='valid', classes=classes)
def get_train_data():
    return train_list

def get_valid_data():
    return valid_list

  0%|          | 0/1380 [00:00<?, ?it/s]

Classes selected: ['PF05257' 'PF06347']


 38%|███▊      | 1233/3222 [00:00<00:00, 12327.43it/s]

Classes selected: ['PF05257' 'PF06347']


100%|██████████| 1380/1380 [00:00<00:00, 22323.90it/s]


In [6]:
DatasetCatalog.register("train", get_train_data)
DatasetCatalog.register("valid", get_valid_data)

In [7]:
# from detectron2.data.datasets import register_coco_instances
# # images used should be at 224x1000 dim
# register_coco_instances("ballon_train", {}, "/home/satish27may/ProteinDomainDetection/data/PfamData/annotations_224/train.json", "/home/satish27may/ProteinDomainDetection/data/PfamData/images_224/")
# register_coco_instances("ballon_val", {}, "/home/satish27may/ProteinDomainDetection/data/PfamData/annotations_224/val.json", "/home/satish27may/ProteinDomainDetection/data/PfamData/images_224/")
# # register_coco_instances("ballon_test", {}, "/home/satish27may/ProteinDomainDetection/data/PfamData/annotations_224//test.json", "/home/satish27may/ProteinDomainDetection/data/PfamData/images_224/")
# balloon_metadata = MetadataCatalog.get("balloon_train")

# ! mkdir /home/satish27may/ProteinDomainDetection/data/PfamData/torch_dataset
# ! trash-put /home/satish27may/ProteinDomainDetection/data/PfamData/torch_dataset/class1
# ! trash-put /home/satish27may/ProteinDomainDetection/data/PfamData/torch_dataset/images_224
# ! mkdir /home/satish27may/ProteinDomainDetection/data/PfamData/torch_dataset/class1
# ! cp  -r /home/satish27may/ProteinDomainDetection/data/PfamData/images_224/  /home/satish27may/ProteinDomainDetection/data/PfamData/torch_dataset/class1/
# #! mv /home/satish27may/ProteinDomainDetection/data/PfamData/torch_dataset/images_224 /home/satish27may/DetectoRS_mmdetect/mmdetection/data/torch_dataset/class1
# def get_mean_std(loader):
#     # var[X] = E[X**2] - E[X]**2
#     channels_sum, channels_sqrd_sum, num_batches = 0, 0, 0

#     for data, _ in tqdm(loader):
#         channels_sum += torch.mean(data, dim=[0, 2, 3])
#         channels_sqrd_sum += torch.mean(data ** 2, dim=[0, 2, 3])
#         num_batches += 1

#     mean = channels_sum / num_batches
#     std = (channels_sqrd_sum / num_batches - mean ** 2) ** 0.5

#     return mean, std


# custom_dataset = datasets.ImageFolder('/home/satish27may/ProteinDomainDetection/data/PfamData/torch_dataset', transform=transforms.ToTensor())
# custom_data_loader = DataLoader(dataset=custom_dataset,batch_size=128)
# data_mean, data_std = get_mean_std(custom_data_loader)

# data_mean = data_mean.numpy().tolist()
# data_std = data_std.numpy().tolist()

# dataset_dicts = DatasetCatalog.get("ballon_train")
# for d in random.sample(dataset_dicts, 3):
#     print(d)
#     img = cv2.imread(d["file_name"])
#     visualizer = Visualizer(img[:, :, ::-1], metadata=balloon_metadata, scale=1.5)
#     bbox = d['annotations'][0]['bbox']
#     out = visualizer.draw_box((bbox[0],bbox[1],bbox[0]+bbox[2],bbox[1]+bbox[3]), edge_color='red', line_style='-')
#     cv2_imshow(out.get_image()[:, :, ::-1])

In [7]:
from detectron2.data import MetadataCatalog
MetadataCatalog.get("train").thing_classes = classes
MetadataCatalog.get("valid").thing_classes = classes

In [8]:
! trash-put /home/satish27may/ProteinDomainDetection/models

In [9]:
from detectron2.engine import DefaultTrainer
from detectron2.config import CfgNode as CN
img_h, img_w = 224, 1000
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("valid",)
#cfg.MODEL.PIXEL_MEAN = data_mean
#cfg.MODEL.PIXEL_STD = data_std
cfg.INPUT.RANDOM_FLIP = "vertical"
cfg.TEST.DETECTIONS_PER_IMAGE = 100


cfg.TEST.AUG.FLIP = False
cfg.DATALOADER.NUM_WORKERS = 8
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 1e-3  # pick a good LR
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
#cfg.MODEL.RETINANET.IOU_THRESHOLDS = [0.4, 0.5]
cfg.SOLVER.MAX_ITER = 300
cfg.MODEL.RETINANET.NUM_CLASSES = len(classes)
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(data_handler.class_names)

cfg.OUTPUT_DIR = '/home/satish27may/ProteinDomainDetection/models'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

Loading config /home/satish27may/anaconda3/envs/detectron2/lib/python3.7/site-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[11/26 04:58:54 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

Skip loading parameter 'head.cls_score.weight' to the model due to incompatible shapes: (720, 256, 3, 3) in the checkpoint but (18, 256, 3, 3) in the model! You might want to double check if this is expected.
Skip loading parameter 'head.cls_score.bias' to the model due to incompatible shapes: (720,) in the checkpoint but (18,) in the model! You might want to double check if this is expected.


[11/26 04:58:56 d2.engine.train_loop]: Starting training from iteration 0


/home/satish27may/anaconda3/envs/detectron2/lib/python3.7/site-packages/fvcore/transforms/transform.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  tensor = torch.from_numpy(np.ascontiguousarray(img))
/home/satish27may/anaconda3/envs/detectron2/lib/python3.7/site-packages/fvcore/transforms/transform.py:433: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writea

[11/26 04:59:11 d2.utils.events]:  eta: 0:03:15  iter: 19  total_loss: 0.7764  loss_cls: 0.5911  loss_box_reg: 0.2191  time: 0.7277  data_time: 0.0290  lr: 1.9784e-05  max_mem: 1164M
[11/26 04:59:25 d2.utils.events]:  eta: 0:03:01  iter: 39  total_loss: 1.204  loss_cls: 0.8831  loss_box_reg: 0.3427  time: 0.7115  data_time: 0.0049  lr: 3.8318e-05  max_mem: 1164M
[11/26 04:59:39 d2.utils.events]:  eta: 0:02:47  iter: 59  total_loss: 0.8387  loss_cls: 0.5997  loss_box_reg: 0.2622  time: 0.7062  data_time: 0.0045  lr: 5.44e-05  max_mem: 1164M
[11/26 04:59:53 d2.utils.events]:  eta: 0:02:33  iter: 79  total_loss: 0.6939  loss_cls: 0.4233  loss_box_reg: 0.2131  time: 0.7033  data_time: 0.0045  lr: 6.7009e-05  max_mem: 1164M
[11/26 05:00:07 d2.utils.events]:  eta: 0:02:19  iter: 99  total_loss: 0.6463  loss_cls: 0.402  loss_box_reg: 0.2439  time: 0.7010  data_time: 0.0047  lr: 7.5377e-05  max_mem: 1164M
[11/26 05:00:21 d2.utils.events]:  eta: 0:02:05  iter: 119  total_loss: 0.5325  loss_cls:

In [10]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("valid", ("bbox",), False, output_dir=cfg.OUTPUT_DIR )
val_loader = build_detection_test_loader(cfg, "valid")
print(trainer.test(cfg, trainer.model, evaluator))

[11/26 05:02:27 d2.evaluation.coco_evaluation]: 'valid' is not registered by `register_coco_instances`. Therefore trying to convert it to COCO format ...
[11/26 05:02:27 d2.data.datasets.coco]: Converting annotations of dataset 'valid' to COCO format ...)
[11/26 05:02:27 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[11/26 05:02:27 d2.data.datasets.coco]: Conversion finished, #images: 1380, #annotations: 1380
[11/26 05:02:27 d2.data.datasets.coco]: Caching COCO format annotations at '/home/satish27may/ProteinDomainDetection/models/valid_coco_format.json' ...
[11/26 05:02:27 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[11/26 05:02:27 d2.data.common]: Serializing 1380 elements to byte tensors and concatenating them all ...
[11/26 05:02:27 d2.data.common]: Serialized dataset takes 0.38 MiB
[11/26 05:02:27 d2.data.dataset_mapper]: [DatasetMapper] Augmen

In [ ]:
predictor = DefaultPredictor(cfg)
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
| category     | AP     | category      | AP     |
|:-------------|:-------|:--------------|:-------|
| CHAP-PF05257 | 55.940 | SH3_4-PF06347 | 21.454 |
 


| category   | AP     | category   | AP     |
|:-----------|:-------|:-----------|:-------|
| PF05257    | 41.624 | PF06347    | 23.025 |




COCOeval_opt.accumulate() finished in 0.62 seconds.
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.343
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.450
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.383
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.201
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.344
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.581
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.730
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.733
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.200
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.733
[11/23 10:26:47 d2.evaluation.coco_evaluation]: Evaluation results for bbox: 
|   AP   |  AP50  |  AP75  |  APs  |  APm   |  APl   |
|:------:|:------:|:------:|:-----:|:------:|:------:|
| 34.336 | 45.023 | 38.327 |  nan  | 20.099 | 34.352 |
[11/23 10:26:47 d2.evaluation.coco_evaluation]: Some metrics cannot be computed and is shown as NaN.
[11/23 10:26:47 d2.evaluation.coco_evaluation]: Per-category bbox AP: 
| category   | AP     | category   | AP     | category   | AP     |
|:-----------|:-------|:-----------|:-------|:-----------|:-------|
| PF01183    | 54.320 | PF05257    | 11.621 | PF00959    | 28.577 |
| PF00182    | 35.231 | PF04965    | 70.866 | PF06347    | 5.398  |